# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 24 2023 1:02PM,258140,10,PRF-53993,Bio-PRF,Released
1,Feb 24 2023 12:59PM,258139,10,Eme-109472,Emerginnova,Released
2,Feb 24 2023 12:59PM,258139,10,Eme-109478,Emerginnova,Released
3,Feb 24 2023 12:25PM,258138,19,OPUS-105182,Opus Life Sciences,Released
4,Feb 24 2023 11:43AM,258134,10,SONSB0002114,"Nextsource Biotechnology, LLC",Released
5,Feb 24 2023 11:34AM,258131,10,0086430550,ISDIN Corporation,Released
6,Feb 24 2023 11:34AM,258131,10,0086430565,ISDIN Corporation,Released
7,Feb 24 2023 11:34AM,258131,10,0086428552,ISDIN Corporation,Released
8,Feb 24 2023 11:34AM,258131,10,0086428553,ISDIN Corporation,Released
9,Feb 24 2023 11:34AM,258131,10,0086428563,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,258131,Released,13
30,258134,Released,1
31,258138,Released,1
32,258139,Released,2
33,258140,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258131,NaN,NaN,13.0
258134,NaN,NaN,1.0
258138,NaN,NaN,1.0
258139,NaN,NaN,2.0
258140,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258069,0.0,3.0,4.0
258071,3.0,11.0,1.0
258072,0.0,0.0,1.0
258075,0.0,0.0,4.0
258079,0.0,9.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258069,0,3,4
258071,3,11,1
258072,0,0,1
258075,0,0,4
258079,0,9,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258069,0,3,4
1,258071,3,11,1
2,258072,0,0,1
3,258075,0,0,4
4,258079,0,9,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258069,,3,4
1,258071,3,11,1
2,258072,,,1
3,258075,,,4
4,258079,,9,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 24 2023 1:02PM,258140,10,Bio-PRF
1,Feb 24 2023 12:59PM,258139,10,Emerginnova
3,Feb 24 2023 12:25PM,258138,19,Opus Life Sciences
4,Feb 24 2023 11:43AM,258134,10,"Nextsource Biotechnology, LLC"
5,Feb 24 2023 11:34AM,258131,10,ISDIN Corporation
18,Feb 24 2023 11:28AM,258130,10,ISDIN Corporation
19,Feb 24 2023 11:28AM,258129,10,"Snap Medical Industries, LLC"
21,Feb 24 2023 11:10AM,258128,12,Hush Hush
22,Feb 24 2023 10:59AM,258127,10,ISDIN Corporation
24,Feb 24 2023 10:46AM,258123,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 24 2023 1:02PM,258140,10,Bio-PRF,,,1
1,Feb 24 2023 12:59PM,258139,10,Emerginnova,,,2
2,Feb 24 2023 12:25PM,258138,19,Opus Life Sciences,,,1
3,Feb 24 2023 11:43AM,258134,10,"Nextsource Biotechnology, LLC",,,1
4,Feb 24 2023 11:34AM,258131,10,ISDIN Corporation,,,13
5,Feb 24 2023 11:28AM,258130,10,ISDIN Corporation,,,1
6,Feb 24 2023 11:28AM,258129,10,"Snap Medical Industries, LLC",,,2
7,Feb 24 2023 11:10AM,258128,12,Hush Hush,,,1
8,Feb 24 2023 10:59AM,258127,10,ISDIN Corporation,,,2
9,Feb 24 2023 10:46AM,258123,10,ISDIN Corporation,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 1:02PM,258140,10,Bio-PRF,1,,
1,Feb 24 2023 12:59PM,258139,10,Emerginnova,2,,
2,Feb 24 2023 12:25PM,258138,19,Opus Life Sciences,1,,
3,Feb 24 2023 11:43AM,258134,10,"Nextsource Biotechnology, LLC",1,,
4,Feb 24 2023 11:34AM,258131,10,ISDIN Corporation,13,,
5,Feb 24 2023 11:28AM,258130,10,ISDIN Corporation,1,,
6,Feb 24 2023 11:28AM,258129,10,"Snap Medical Industries, LLC",2,,
7,Feb 24 2023 11:10AM,258128,12,Hush Hush,1,,
8,Feb 24 2023 10:59AM,258127,10,ISDIN Corporation,2,,
9,Feb 24 2023 10:46AM,258123,10,ISDIN Corporation,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 1:02PM,258140,10,Bio-PRF,1,,
1,Feb 24 2023 12:59PM,258139,10,Emerginnova,2,,
2,Feb 24 2023 12:25PM,258138,19,Opus Life Sciences,1,,
3,Feb 24 2023 11:43AM,258134,10,"Nextsource Biotechnology, LLC",1,,
4,Feb 24 2023 11:34AM,258131,10,ISDIN Corporation,13,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 1:02PM,258140,10,Bio-PRF,1,NaN,NaN
1,Feb 24 2023 12:59PM,258139,10,Emerginnova,2,NaN,NaN
2,Feb 24 2023 12:25PM,258138,19,Opus Life Sciences,1,NaN,NaN
3,Feb 24 2023 11:43AM,258134,10,"Nextsource Biotechnology, LLC",1,NaN,NaN
4,Feb 24 2023 11:34AM,258131,10,ISDIN Corporation,13,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 1:02PM,258140,10,Bio-PRF,1,0.0,0.0
1,Feb 24 2023 12:59PM,258139,10,Emerginnova,2,0.0,0.0
2,Feb 24 2023 12:25PM,258138,19,Opus Life Sciences,1,0.0,0.0
3,Feb 24 2023 11:43AM,258134,10,"Nextsource Biotechnology, LLC",1,0.0,0.0
4,Feb 24 2023 11:34AM,258131,10,ISDIN Corporation,13,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4645997,137,12.0,0.0
12,258128,1,0.0,0.0
15,516192,6,0.0,0.0
19,1290482,10,37.0,11.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4645997,137,12.0,0.0
1,12,258128,1,0.0,0.0
2,15,516192,6,0.0,0.0
3,19,1290482,10,37.0,11.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,137,12.0,0.0
1,12,1,0.0,0.0
2,15,6,0.0,0.0
3,19,10,37.0,11.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,137.0
1,12,Released,1.0
2,15,Released,6.0
3,19,Released,10.0
4,10,Executing,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19
Status,,,,
Completed,0.0,0.0,0.0,11.0
Executing,12.0,0.0,0.0,37.0
Released,137.0,1.0,6.0,10.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19
0,Completed,0.0,0.0,0.0,11.0
1,Executing,12.0,0.0,0.0,37.0
2,Released,137.0,1.0,6.0,10.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19
0,Completed,0.0,0.0,0.0,11.0
1,Executing,12.0,0.0,0.0,37.0
2,Released,137.0,1.0,6.0,10.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()